In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
""" Load the data """

df = pd.read_csv('../data/raw/raw_nyc_traffic_data.csv')  # Update with your data source

# Rename columns for clarity
df.rename(columns={'Yr': 'year', 'M': 'month', 'D': 'day', 'HH': 'hour', 'MM': 'minute'}, inplace=True)

# Create timestamp
df['timestamp'] = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute']])

# Handle outliers: Remove top 1% of vol to reduce skew
df = df[df['Vol'] <= df['Vol'].quantile(0.99)]

In [7]:
""" Feature Engineering """

# Derived time features
df['hour'] = df['timestamp'].dt.hour
df['dayofweek'] = df['timestamp'].dt.dayofweek
df['is_weekend'] = df['dayofweek'].isin([5, 6]).astype(int)
df['is_rush_hour'] = df['hour'].isin([7, 8, 9, 16, 17, 18]).astype(int)
df['month'] = df['timestamp'].dt.month
df['season'] = df['timestamp'].dt.month % 12 // 3  # 0: Winter, 1: Spring, etc.

# Lagged feature (previous hour's volume by segment, borough, direction)
df['vol_lag1'] = df.groupby(['SegmentID', 'Boro', 'Direction'])['Vol'].shift(1)

# Log-transform target variable
df['log_vol'] = np.log1p(df['Vol'])  # log(1 + x) to handle zeros

# Drop rows with NaN values from lagged features
df = df.dropna()

# Define features
numerical = ['hour', 'minute', 'dayofweek', 'is_weekend', 'is_rush_hour', 'month', 'season', 'vol_lag1']
categorical = ['Boro', 'Direction', 'SegmentID']

/var/folders/px/j7p2_nnn6h339lc16xg05qbw0000gn/T/ipykernel_46160/3685360349.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hour'] = df['timestamp'].dt.hour
/var/folders/px/j7p2_nnn6h339lc16xg05qbw0000gn/T/ipykernel_46160/3685360349.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayofweek'] = df['timestamp'].dt.dayofweek
/var/folders/px/j7p2_nnn6h339lc16xg05qbw0000gn/T/ipykernel_46160/3685360349.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

In [8]:
""" Train-Test Split """

# Define X and y
X = df[numerical + categorical]
y = df['log_vol']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
""" Preprocessing pipeline with RobustScaler for numerical features """

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical),
        ('num', RobustScaler(), numerical)
    ]
)

In [10]:
""" Random Forest Modeling with tuning """

# Random Forest pipeline
rf_model = make_pipeline(
    preprocessor,
    RandomForestRegressor(random_state=42, n_jobs=-1)
)

# Hyperparameter tuning
param_grid = {
    'randomforestregressor__n_estimators': [100, 200],
    'randomforestregressor__max_depth': [10, 15, None],
    'randomforestregressor__min_samples_split': [2, 5],
    'randomforestregressor__min_samples_leaf': [1, 2]
}
grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best model and predictions
print(f"Best parameters: {grid_search.best_params_}")
y_pred_log = grid_search.predict(X_test)
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)
rf_mae = mean_absolute_error(y_true, y_pred)
print(f"Random Forest MAE: {rf_mae:.2f}")

/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:960: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 949, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 288, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 388, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

KeyboardInterrupt: 

In [ ]:
""" XGBoost Model """

# XGBoost pipeline
xgb_model = make_pipeline(
    preprocessor,
    XGBRegressor(objective='reg:absoluteerror', random_state=42, n_jobs=-1)
)

# Train and predict
xgb_model.fit(X_train, y_train, xgboostregressor__sample_weight=1 / (1 + np.expm1(y_train)))  # Weight lower vol
y_pred_log = xgb_model.predict(X_test)
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)
xgb_mae = mean_absolute_error(y_true, y_pred)
print(f"XGBoost MAE: {xgb_mae:.2f}")

In [ ]:
""" Cross Validation """

# Cross-validation for Random Forest
rf_scores = cross_val_score(rf_model, X, y, cv=5, scoring='neg_mean_absolute_error')
print(f"Random Forest CV MAE (log scale): {-rf_scores.mean():.4f}")
print(f"Estimated CV MAE (original scale): {np.expm1(-rf_scores.mean()):.2f}")

# Cross-validation for XGBoost
xgb_scores = cross_val_score(xgb_model, X, y, cv=5, scoring='neg_mean_absolute_error')
print(f"XGBoost CV MAE (log scale): {-xgb_scores.mean():.4f}")
print(f"Estimated CV MAE (original scale): {np.expm1(-xgb_scores.mean()):.2f}")